### Downloading and cleaning Toronoto Dataset.

#### Importing necessary packages

In [4]:
import lxml
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests

#### Reading the data directly from the html into a dataframe.

In [5]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
data = pd.read_html(url)
neighborhoods = data[0]

#Assigning column names.
column_names = ['PostalCode', 'Borough', 'Neighborhood']
neighborhoods.columns = column_names
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Removing rows with unassigned borough by changing it to NaN and droping rows containing NaN. 

In [6]:
neighborhoods['Borough'] = neighborhoods['Borough'].replace('Not assigned', np.NaN)
neighborhoods = neighborhoods.dropna().reset_index(drop=True)

In [7]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### No occurances of not assigned in Neighborhood column, so no need to cast any Borough values into this column.

In [8]:
(neighborhoods['Neighborhood'] == 'Not assigned').sum()

0

In [9]:
neighborhoods.shape

(103, 3)